<a href="https://colab.research.google.com/github/MishraShardendu22/Transformers/blob/main/Translate_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets sentencepiece accelerate evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.8 MB/s eta 0:00:00


# Explaination (ai help)

```python
dataset = dataset.train_test_split(test_size=0.1)
```

### What it does

It splits your dataset into two parts:

* **90% → training set**
* **10% → validation (test) set**

Since you selected **30,000 samples**:

* 27,000 → used to train the model
* 3,000 → used to evaluate model performance

---

### Why this is required

During training:

* Model learns on the **train set**
* After each epoch, performance is checked on the **validation set**
* Prevents overfitting
* Lets you measure BLEU score properly

---

### What `print(dataset)` shows

You will see something like:

```
DatasetDict({
    train: Dataset({
        features: ...
        num_rows: 27000
    })
    test: Dataset({
        features: ...
        num_rows: 3000
    })
})
```

In [ ]:
from datasets import load_dataset

# Correct dataset name
dataset = load_dataset("cfilt/iitb-english-hindi")

# Shuffle and take 1,000,000 samples
dataset = dataset["train"].shuffle(seed=42).select(range(1_000_000))

# Train-validation split
dataset = dataset.train_test_split(test_size=0.1)

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 900000
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 100000
    })
})


In [ ]:
from transformers import (
    AutoTokenizer,
    EncoderDecoderModel,
    BertConfig
)

# We still use tokenizer (vocab only, not weights)
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

# Create encoder config
encoder_config = BertConfig(
    vocab_size=tokenizer.vocab_size,
    hidden_size=512,
    num_hidden_layers=6,
    num_attention_heads=8,
    intermediate_size=2048,
    max_position_embeddings=512,
)

# Create decoder config
decoder_config = BertConfig(
    vocab_size=tokenizer.vocab_size,
    hidden_size=512,
    num_hidden_layers=6,
    num_attention_heads=8,
    intermediate_size=2048,
    is_decoder=True,
    add_cross_attention=True,
    max_position_embeddings=512,
)

# Build model
model = EncoderDecoderModel.from_encoder_decoder_configs(
    encoder_config,
    decoder_config
)

model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.pad_token_id = tokenizer.pad_token_id

print("Scratch Transformer created")

In [ ]:
def preprocess_function(examples):
    inputs = [x["hi"] for x in examples["translation"]]
    targets = [x["en"] for x in examples["translation"]]

    model_inputs = tokenizer(
        inputs,
        max_length=128,
        padding="max_length",
        truncation=True,
    )

    labels = tokenizer(
        targets,
        max_length=128,
        padding="max_length",
        truncation=True,
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset["train"].column_names,
    num_proc=2
)

print(tokenized_dataset)

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model
)

training_args = Seq2SeqTrainingArguments(
    output_dir="./scratch-hi-en",
    evaluation_strategy="steps",
    save_strategy="steps",
    logging_steps=1000,
    save_steps=5000,
    eval_steps=5000,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=5e-4,
    weight_decay=0.01,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    save_total_limit=2
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

print("Trainer ready")

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model
)

training_args = Seq2SeqTrainingArguments(
    output_dir="./scratch-hi-en",
    eval_strategy="steps",
    save_strategy="steps",
    logging_steps=1000,
    save_steps=5000,
    eval_steps=5000,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=5e-4,
    weight_decay=0.01,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    save_total_limit=2
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
)

print("Trainer ready")

In [ ]:
trainer.train()

In [ ]:
text = "मुझे मशीन लर्निंग पसंद है"

inputs = tokenizer(
    text,
    return_tensors="pt",
    max_length=128,
    truncation=True
)

outputs = model.generate(
    inputs["input_ids"].to(model.device),
    attention_mask=inputs["attention_mask"].to(model.device),
    max_length=128
)

translation = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Translation:", translation)